In [ ]:
# libraries we'll use
import numpy as np
import matplotlib.pyplot as plt

# MD code

Consider a system of atoms interacting with a Lennard Jones potential function which acts between pairs of particles.

$E_{LJ} (\mathbf{r}) = 4 \epsilon \left[\left( \frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r} \right)^6 \right]$

in one dimension. Where $r$ is $ |r_i - r_j|$.

The force on the particles is given by
$F = -\nabla E$

## Parameters and starting conditions

now to setup the parameters

In [ ]:
nsteps = 1000
dt = 0.1
mass = 1.0
output_freq = 100

We store positions etc as (N x 3) D arrays. First index is the atom number (starting from zero). The second index is the set of  cartesian directions for that atom.

To add remove atoms you just add or remove a set of coordinates. The are python arrays [xi,yi,zi] and you need a comma between them. The outer []s make an array or arrays (2D array). 

In [ ]:
#atoms = np.array([
#    [0.0, 0.0, 0.0],
#    [0.0,0.0,3.2],
#    [0.0,0.0,6.4]]
#)
# set up the initial position of the atoms - we could do the same for velocities
a = 3.1
atoms = np.zeros((32,3))
for i in range(32):
    atoms[i,2] = i*a
atoms

natoms = len(atoms)

## The rest of the code actually does the calculation.

We are going to store all the information about our system in a dictionary.

Using a class would be another option.

In [ ]:
model = dict(natoms = natoms,
             atoms = atoms,
             vel = np.zeros((natoms,3)), 
             forcestp1 = np.zeros((natoms,3)),  
             forcest = np.zeros((natoms,3)),  
             pot_energy = [], 
             KE = [],
             traj = [],
             temp_pos = np.zeros((natoms,3)),
             temp_vel = np.zeros((natoms,3)),
             output_freq = output_freq,
             integrator = 'vverlet'
)

You can access the value of a dictionary using its key. 

In [ ]:
model['atoms'];

In [ ]:
model['vel'];

Define our energy and force functions

In [ ]:
def energyij(model, atomi, atomj, sigma=3, epsilon=0.1):
    """calculates the Lennard Jones potential energy of two particles
    model - model data structure
    atomi - index of atom i
    atomj - index of atom j
    """
    r = np.linalg.norm(model['atoms'][atomi] - model['atoms'][atomj])
    u = 4*epsilon*((sigma/r)**12-(sigma/r)**6)
    return u

def forceij(model, atomi, atomj, sigma=3, epsilon=0.1):
    """calculates the Lennard Jones force between two particles
    model - model data structure
    atomi - index of atom i
    atomj - index of atom j
    returns forces on atom i, fi,  and atom j, fj
    """
    #get magnitude and direction of forces - use Newton's 3rd law for fi = -fj 
    r = np.linalg.norm(model['atoms'][atomi] - model['atoms'][atomj])
    dir = (model['atoms'][atomi] - model['atoms'][atomj])/r
    #calc force magnitude
    fr = -4*epsilon*(12*sigma**12*r**(-13)-6*sigma**6*r**(-7))
    return -fr*dir, fr*dir

In [ ]:
# add our potential energy model into the model
# doing it like this makes it easy to swap in new force routines

model['potential'] = energyij
model['force'] = forceij

In [ ]:
def thermostat(model):

    # langevin thermostat on all degrees of freedom
    gamma = 0.1
    kbT = 0.5
    sigma = np.sqrt(2*gamma*kbT)
    for atomi in range(model['natoms']):
        model['forcestp1'][atomi] += -gamma*model['vel'][atomi] + sigma*np.random.normal(0, sigma)
    

If we have pair potential forces only dependent on the distance, $r_{ij}$ , between a pair of particles we have that
$$
\mathbf{F_i} = \frac{\partial U}{\partial r_{ij}} \cdot \frac{\mathbf{r_{ij}}}{r_{ij}}
$$
and $\mathbf{F_j} = -\mathbf{F_i}$ (Newton's law or due to the change in sign of $\mathbf{r_{ji}}$). 

If we have have multiple particles we should add up all the unique pairs of interactions: 

In [ ]:
def calcForces(model):
    energy = 0
    # remember to reset forces at each step
    model['forcestp1'][:] = 0.0
    for atomi in range(model['natoms']):
        #avoid double counting
        for atomj in range(atomi+1, natoms):
            energy += model['potential'](model, atomi, atomj)
            fi, fj = model['force'](model,atomi,atomj)
            model['forcestp1'][atomi] += fi
            model['forcestp1'][atomj] += fj
    #print(model['forcestp1'])

    do_thermostat = False
    if do_thermostat:
        thermostat(model)
        
    return energy

Now the integration routines

In [ ]:
def integrate(model):
    if model['integrator'] == 'Euler':
        pot_energy = calcForces(model)
        model['atoms'] += model['vel']*dt + 0.5*model['forcestp1']/mass*dt**2
        model['vel'] += model['forcestp1']/mass*dt

    elif model['integrator'] == 'vverlet':
        model['forcest'] =  np.copy(model['forcestp1'])
        model['atoms'] += model['vel']*dt + 0.5*model['forcest']/mass*dt**2
        pot_energy = calcForces(model)
        model['vel'] += 0.5*(model['forcest'] + model['forcestp1'])/mass*dt            

    else:
        print('no known integrator! falling back on Euler')
        pot_energy = calcForces(model)
        model['atoms'] += model['vel']*dt + 0.5*model['forcestp1']/mass*dt**2
        model['vel'] += model['forcestp1']/mass*dt
            
    # calculate the Kinetic Energy
    KE = 0    
    for i in range(model['natoms']):
        KE += np.dot(model['vel'][i],model['vel'][i])

    return pot_energy, KE

## Run MD

Now the actual main MD loop that integrates in time!

In [ ]:
#calc energy / forces
calcForces(model)
for step in range(nsteps):

    # integrate
    pot_energy, KE = integrate(model)
    #print(model['forcestp1'])
    
    # save data
    model['pot_energy'].append(pot_energy)
    model['KE'].append(0.5*mass*KE)
    
    if not step%model['output_freq']:
        print("step {}".format(step))
        # need to copy otherwise we just get a pointer to the latest temp_pos
        model['traj'].append(np.copy(model['atoms']))    

## Analyse our results

At the end we have stored some information and can plot it out

In [ ]:
plt.plot(model['pot_energy'], label='PE')
plt.plot(np.array(model['KE']), label='KE')
plt.plot(model['pot_energy'] + np.array(model['KE']), label='total')
plt.legend()
plt.savefig('test.jpg')

In [ ]:
np.mean(model['KE'])

In [ ]:
plt.plot(np.array(model['KE']))

Of course a lot more can be done.

In [ ]:
model['traj'];